<h2> Task 10. Implement Genetic Algorithm (GA) Optimization </h2>
The goal of this task is to implement a Genetic Algorithm to solve an optimization problem. GA can optimize continuous (numeric) and discrete (categorical) problems. For example

*   Rastrigin, an example of continous objective function you have implemented above. But use at least 10 dimnesions. Check code bellow.
*   Knapsack Problem, the Knapsack Problem is a classic optimization problem where the goal is to maximize the total value of items that can be placed into a knapsack without exceeding its weight capacity

<p>Let's start with implementation</p>

1.   Implement a selection mechanisms: tournament selection, roulette wheel selection. Their purpose is to choose individuals based on their fitness values
2.   Implement crossover techniques to combine the genetic information of two parent individuals to create offspring. Implement, single-point crossover, 2-point crossover and uniform crossover.
3.   Implement mutation: Introduce random mutations in offspring to maintain genetic diversity. Implement gaussian mutation for numeric problems and implement Flip-bit mutation for discrete problems.
4.   Create a main loop which performs the optimization
5.   Create an inner loop which creates new population based on current population
6.   Create encoding for Knapsack problem. For example, knapsack containing only first item (out of 4) will be encoded as [1,0,0,0]. Knapsack containing second and fourth item will be encoded as [0,1,0,1], etc...
7. Finish the fitness function for Knapsack problem
8. Run on both problems and get results


In [1]:
import numpy as np

def rastrigin_high_dimen(x):
    A = 10
    n = len(x)
    if n < 10:
        print("Use at least 10 dimensions in rastrigin_high_dimen")
        return -1
    return A * n + np.sum(x**2 - A * np.cos(2*np.pi*x))

def selection_tournament(population, fitness_scores, k=3):
    parents = []
    N = len(population)
    fitness_scores = np.asarray(fitness_scores)
    for _ in range(N):
        tournament_size = min(k, N)
        replace_needed = tournament_size >= N
        idxs = np.random.choice(N, size=tournament_size, replace=replace_needed)
        best = idxs[np.argmax(fitness_scores[idxs])]
        parents.append(population[best].copy())
    return parents

def selection_roulette(population, fitness_scores):
    f = np.asarray(fitness_scores, dtype=float)
    f = f - f.min() + 1e-9  
    probs = f / f.sum()
    idxs = np.random.choice(len(population), size=len(population), p=probs)
    return [population[i].copy() for i in idxs]

def crossover_single(parent1, parent2):
    cut = np.random.randint(1, len(parent1))
    c1 = np.concatenate([parent1[:cut], parent2[cut:]])
    c2 = np.concatenate([parent2[:cut], parent1[cut:]])
    return c1, c2

def crossover_two(parent1, parent2):
    a, b = sorted(np.random.choice(np.arange(1, len(parent1)), size=2, replace=False))
    c1 = parent1.copy(); c2 = parent2.copy()
    c1[a:b], c2[a:b] = parent2[a:b], parent1[a:b]
    return c1, c2

def crossover_uniform(parent1, parent2, p=0.5):
    mask = np.random.rand(len(parent1)) < p
    c1 = np.where(mask, parent1, parent2)
    c2 = np.where(mask, parent2, parent1)
    return c1, c2

def mutate_gaussian(individual, rate=0.12, sigma=0.3, bounds=(-5.12, 5.12)):
    out = individual.copy()
    mask = np.random.rand(len(out)) < rate
    out[mask] = out[mask] + np.random.normal(0, sigma, size=mask.sum())
    return np.clip(out, bounds[0], bounds[1])

dim = 12
bounds = (-5.12, 5.12)
pop_size = 60
generations = 120

population = [np.random.uniform(bounds[0], bounds[1], dim) for _ in range(pop_size)]

for gen in range(generations):
    
    obj_vals = np.array([rastrigin_high_dimen(ind) for ind in population])
    fitness_scores = 1.0 / (1.0 + obj_vals) 

    parents = selection_tournament(population, fitness_scores, k=3)

    next_pop = []
    for i in range(0, len(parents), 2):
        p1 = parents[i]
        p2 = parents[i+1] if i+1 < len(parents) else parents[0]

        if i % 4 == 0:
            c1, c2 = crossover_uniform(p1, p2, p=0.5)
        elif i % 4 == 2:
            c1, c2 = crossover_two(p1, p2)
        else:
            c1, c2 = crossover_single(p1, p2)

        c1 = mutate_gaussian(c1, rate=0.12, sigma=0.3, bounds=bounds)
        c2 = mutate_gaussian(c2, rate=0.12, sigma=0.3, bounds=bounds)

        next_pop.extend([c1, c2])

    population = next_pop[:pop_size]

final_objs = np.array([rastrigin_high_dimen(ind) for ind in population])
best_idx = np.argmin(final_objs)
best_ind = population[best_idx]
print("Rastrigin result (min value):", final_objs[best_idx])
print("Best x (first 6 dims):", np.round(best_ind[:6], 4))

x = np.array([1.2, 2.3, 20, 3, 3.2, 5.4, 42.1, 2, 1, 2, 4, 5])
print(f'Rastrigin function value at {x}: {rastrigin_high_dimen(x)}')


Rastrigin result (min value): 13.397729542062507
Best x (first 6 dims): [ 0.9574  1.975   0.0166  0.0064 -0.9657 -0.0877]
Rastrigin function value at [ 1.2  2.3 20.   3.   3.2  5.4 42.1  2.   1.   2.   4.   5. ]: 2324.4498300562504


In [2]:
import numpy as np

knapsack_items = {
    'item1': {'value': 60, 'weight': 10},
    'item2': {'value': 100, 'weight': 20},
    'item3': {'value': 120, 'weight': 30},
    'item4': {'value': 80, 'weight': 15},
    'item5': {'value': 40, 'weight': 5},
    'item6': {'value': 70, 'weight': 25},
    'item7': {'value': 90, 'weight': 35},
    'item8': {'value': 150, 'weight': 40},
    'item9': {'value': 200, 'weight': 50},
    'item10': {'value': 30, 'weight': 10}
}
capacity = 100

names = list(knapsack_items.keys())
values = np.array([knapsack_items[k]['value'] for k in names], dtype=float)
weights = np.array([knapsack_items[k]['weight'] for k in names], dtype=float)
n_items = len(names)

def fitness_knapsack(individual):
    total_value = float(np.dot(individual, values))
    total_weight = float(np.dot(individual, weights))
    return total_value if total_weight <= capacity else 0.0

def mutate_flip_bits(bits, rate=0.05):
    out = bits.copy()
    mask = np.random.rand(len(bits)) < rate
    out[mask] = 1 - out[mask]
    return out

def selection_roulette(pop, fits):
    f = np.asarray(fits, dtype=float)
    f = f - f.min() + 1e-9
    probs = f / f.sum()
    idxs = np.random.choice(len(pop), size=len(pop), p=probs)
    return [pop[i].copy() for i in idxs]

def crossover_two(parent1, parent2):
    a, b = sorted(np.random.choice(np.arange(1, len(parent1)), size=2, replace=False))
    c1 = parent1.copy(); c2 = parent2.copy()
    c1[a:b], c2[a:b] = parent2[a:b], parent1[a:b]
    return c1, c2

def crossover_uniform(p1, p2, p=0.5):
    mask = np.random.rand(len(p1)) < p
    return np.where(mask, p1, p2), np.where(mask, p2, p1)

pop_size = 80
generations = 150

population = [np.random.randint(0, 2, size=n_items, dtype=int) for _ in range(pop_size)]

for gen in range(generations):
    fits = np.array([fitness_knapsack(ind) for ind in population])

    parents = selection_roulette(population, fits)

    next_pop = []
    for i in range(0, len(parents), 2):
        p1 = parents[i]
        p2 = parents[i+1] if i+1 < len(parents) else parents[0]

        if (i // 2) % 2 == 0:
            c1, c2 = crossover_two(p1, p2)
        else:
            c1, c2 = crossover_uniform(p1, p2, p=0.5)

        c1 = mutate_flip_bits(c1, rate=0.05)
        c2 = mutate_flip_bits(c2, rate=0.05)

        next_pop.extend([np.array(c1, dtype=int), np.array(c2, dtype=int)])

    population = next_pop[:pop_size]

fits = np.array([fitness_knapsack(ind) for ind in population])
best_idx = np.argmax(fits)
best_bits = population[best_idx]
print("Knapsack result (max value):", fits[best_idx])
print("Items selected bits:", best_bits)
chosen = [n for n, b in zip(names, best_bits) if b == 1]
print("Chosen items:", chosen, "| Total weight:", np.dot(best_bits, weights))


Knapsack result (max value): 480.0
Items selected bits: [1 1 0 1 1 0 0 0 1 0]
Chosen items: ['item1', 'item2', 'item4', 'item5', 'item9'] | Total weight: 100.0


<h2> Task 11. Experiment with Genetic Algorithm (GA) Optimization Hyperparameters </h2>
Identify key hyperparameters to experiment with, such as:

*   Population Size: The number of individuals in the population
*   Crossover type: Single point vs 2-point vs uniform
*   Mutation Rate: The probability of mutating a chromosome.
*   Selection Method: The method used for selecting parents from the populatio
<p> 1.) Run the Genetic Algorithm with different combinations of population size, crossover type, mutation rate, and selection methods.
For each combination, record metrics such as the best fitness value found and the number of generations required to converge to a solution.
</p>
<p> 2.) Report hyperparameters which will make GA never converge. Why?</p>

In [3]:
import numpy as np

_selection_map = {
    "tournament": selection_tournament,   
    "roulette":   selection_roulette      
}
_crossover_map = {
    "single":   crossover_single,
    "two":      crossover_two,
    "uniform":  crossover_uniform
}

def run_ga_trial(pop_size, crossover_type, mutation_rate, selection_method,
                 problem="knapsack", gens=200, dim=12, bounds=(-5.12, 5.12),
                 tol=1e-2, window=15, seed=None):
    
    if seed is not None:
        np.random.seed(seed)

    # Initialize population
    if problem == "knapsack":
        n = len(values)
        population = [np.random.randint(0, 2, size=n, dtype=int) for _ in range(pop_size)]
        mutate = lambda ind: mutate_flip_bits(ind, rate=mutation_rate)
    else:
        population = [np.random.uniform(bounds[0], bounds[1], size=dim) for _ in range(pop_size)]
        mutate = lambda ind: mutate_gaussian(ind, rate=mutation_rate, sigma=0.3, bounds=bounds)

    cx = _crossover_map[crossover_type]
    best_history = []
    conv_gen = gens

    for g in range(gens):
        if problem == "knapsack":
            scores = np.array([fitness_knapsack(ind) for ind in population], dtype=float)
            parents = _selection_map[selection_method](population, scores, k=3) if selection_method == "tournament" else _selection_map[selection_method](population, scores)
            best_now = float(scores.max())
        else:
            objs = np.array([rastrigin_high_dimen(ind) for ind in population], dtype=float)
            fits = 1.0 / (1.0 + objs)
            parents = _selection_map[selection_method](population, fits, k=3) if selection_method == "tournament" else _selection_map[selection_method](population, fits)
            best_now = float(objs.min())
        
        best_history.append(best_now)

        # Check convergence
        if g >= window:
            window_vals = best_history[-window:]
            if (max(window_vals) - min(window_vals)) <= tol and conv_gen == gens:
                conv_gen = g

        # Create offspring
        next_pop = []
        for i in range(0, len(parents), 2):
            p1 = parents[i]
            p2 = parents[(i+1) % len(parents)]
            c1, c2 = cx(p1, p2)
            c1 = mutate(c1)
            c2 = mutate(c2)
            next_pop.extend([c1, c2])

        population = next_pop[:pop_size]

    if problem == "knapsack":
        final = float(max(fitness_knapsack(ind) for ind in population))
    else:
        final = float(min(rastrigin_high_dimen(ind) for ind in population))

    return final, conv_gen

# Experiment configurations
experiments = [
    {"pop_size": 30,  "crossover": "single",   "mut_rate": 0.10, "selection": "tournament"},
    {"pop_size": 100, "crossover": "uniform",  "mut_rate": 0.05, "selection": "roulette"},
    {"pop_size": 50,  "crossover": "two",      "mut_rate": 0.20, "selection": "tournament"},
    {"pop_size": 20,  "crossover": "single",   "mut_rate": 0.01, "selection": "roulette"},
    {"pop_size": 15,  "crossover": "uniform",  "mut_rate": 0.40, "selection": "tournament"},
]

# Knapsack experiments
print("KNAPSACK EXPERIMENTS:")
print("PopSize | Crossover  | MutRate | Selection   |  BestFitness | Converged@Gen")
print("-"*78)
for exp in experiments:
    runs = []
    for r in range(5):
        best, conv = run_ga_trial(
            pop_size=exp["pop_size"],
            crossover_type=exp["crossover"],
            mutation_rate=exp["mut_rate"],
            selection_method=exp["selection"],
            problem="knapsack",
            gens=200,
            seed=r
        )
        runs.append((best, conv))
    avg_best = float(np.mean([x[0] for x in runs]))
    avg_conv = float(np.mean([x[1] for x in runs]))
    print(f"{exp['pop_size']:6} | {exp['crossover']:10} | {exp['mut_rate']:7.2f} | {exp['selection']:10} |"
          f" {avg_best:12.2f} | {avg_conv:13.1f}")

# Rastrigin experiments
print("\nRASTRIGIN EXPERIMENTS:")
print("PopSize | Crossover  | MutRate | Selection   |   BestValue  | Converged@Gen")
print("-"*78)
for exp in experiments:
    runs = []
    for r in range(5):
        best, conv = run_ga_trial(
            pop_size=exp["pop_size"],
            crossover_type=exp["crossover"],
            mutation_rate=exp["mut_rate"],
            selection_method=exp["selection"],
            problem="rastrigin",
            gens=200,
            dim=12,
            seed=r
        )
        runs.append((best, conv))
    avg_best = float(np.mean([x[0] for x in runs]))
    avg_conv = float(np.mean([x[1] for x in runs]))
    print(f"{exp['pop_size']:6} | {exp['crossover']:10} | {exp['mut_rate']:7.2f} | {exp['selection']:10} |"
          f" {avg_best:12.4f} | {avg_conv:13.1f}")

print("\nNON-CONVERGENCE ANALYSIS:")
print("-" * 50)

problematic_configs = [
    {"pop_size": 5, "mut_rate": 0.0, "problem": "knapsack"},      # No diversity
    {"pop_size": 100, "mut_rate": 0.8, "problem": "knapsack"},   # Too much mutation
    {"pop_size": 2, "mut_rate": 0.1, "problem": "rastrigin"},    # Too small population
    {"pop_size": 50, "mut_rate": 0.0, "problem": "rastrigin"}    # No exploration
]

for config in problematic_configs:
    best, conv = run_ga_trial(
        pop_size=config["pop_size"], 
        crossover_type="uniform", 
        mutation_rate=config["mut_rate"],
        selection_method="tournament",
        problem=config["problem"],
        gens=100,
        seed=42
    )
    print(f"Pop={config['pop_size']}, Mut={config['mut_rate']}, {config['problem']}: {best:.2f}")


KNAPSACK EXPERIMENTS:
PopSize | Crossover  | MutRate | Selection   |  BestFitness | Converged@Gen
------------------------------------------------------------------------------
    30 | single     |    0.10 | tournament |       446.00 |          94.4
   100 | uniform    |    0.05 | roulette   |       460.00 |         143.6
    50 | two        |    0.20 | tournament |       436.00 |         200.0
    20 | single     |    0.01 | roulette   |       426.00 |          29.0
    15 | uniform    |    0.40 | tournament |       396.00 |         200.0

RASTRIGIN EXPERIMENTS:
PopSize | Crossover  | MutRate | Selection   |   BestValue  | Converged@Gen
------------------------------------------------------------------------------
    30 | single     |    0.10 | tournament |      27.0266 |         200.0
   100 | uniform    |    0.05 | roulette   |       6.7505 |         200.0
    50 | two        |    0.20 | tournament |      25.8728 |         200.0
    20 | single     |    0.01 | roulette   |      52

## Hyperparameters that Prevent GA Convergence

**Non-convergent configurations:**
- **Population ≤ 5:** Insufficient diversity
- **Mutation = 0:** No exploration, trapped in local optima  
- **Mutation ≥ 0.8:** Destroys good solutions too quickly
- **Population ≤ 2:** No meaningful crossover possible

**Why they fail:** These parameters eliminate the exploration-exploitation balance needed for evolutionary search. Small populations lack diversity, zero mutation prevents escape from local optima, and excessive mutation destroys beneficial patterns faster than they form.

**Working ranges:** Population 30-100, Mutation 0.01-0.20

<h2> Task 12. Use pyGAD library </h2>
The pyGAD library is a powerful tool for implementing Genetic Algorithms in Python. Check their documentation: https://pygad.readthedocs.io/en/latest/

1.   pip install pygad
2.   Solve Knapsack problem defined above
3.   Solve rastrigin_high_dimen problem defined above
4.   Explore pyGAD hyperparameters


In [4]:
!pip install pygad

In [5]:
import pygad
import numpy as np

def fitness_knapsack_pygad(ga_instance, solution, solution_idx):
    sol = np.array(solution, dtype=int)
    total_value = float(sol @ values)
    total_weight = float(sol @ weights)
    overflow = max(0.0, total_weight - capacity)
    penalty = 1000.0 * overflow
    return total_value - penalty

def fitness_rastrigin_pygad(ga_instance, solution, solution_idx):
    x = np.array(solution, dtype=float)
    return -rastrigin_high_dimen(x)

# Knapsack GA
ga_knap = pygad.GA(
    num_generations=100,
    sol_per_pop=50,
    num_parents_mating=20,
    fitness_func=fitness_knapsack_pygad,
    num_genes=n_items,
    gene_type=int,
    gene_space=[0, 1],
    parent_selection_type="tournament",
    K_tournament=3,
    crossover_type="two_points",
    mutation_probability=0.05,
    keep_parents=2,
    suppress_warnings=True
)

ga_knap.run()
sol_knap, fit_knap, _ = ga_knap.best_solution()
print(f"Knapsack Best Fitness: {fit_knap}")

# Rastrigin GA  
ga_rastr = pygad.GA(
    num_generations=150,
    sol_per_pop=80,
    num_parents_mating=30,
    fitness_func=fitness_rastrigin_pygad,
    num_genes=12,
    gene_type=float,
    gene_space={'low': -5.12, 'high': 5.12},
    parent_selection_type="tournament",
    K_tournament=3,
    crossover_type="uniform",
    mutation_probability=0.10,
    keep_parents=4,
    suppress_warnings=True
)

ga_rastr.run()
sol_rastr, fit_rastr, _ = ga_rastr.best_solution()
print(f"Rastrigin Best Fitness: {fit_rastr} (Rastrigin value: {-fit_rastr})")

print("\nHyperparameter exploration:")

for pop_size in [20, 50, 100]:
    ga = pygad.GA(
        num_generations=50,
        sol_per_pop=pop_size,
        num_parents_mating=10,
        fitness_func=fitness_knapsack_pygad,
        num_genes=n_items,
        gene_type=int,
        gene_space=[0, 1],
        parent_selection_type="tournament",
        suppress_warnings=True
    )
    ga.run()
    _, fitness, _ = ga.best_solution()
    print(f"Population {pop_size}: {fitness}")

for mut_rate in [0.05, 0.15, 0.25]:
    ga = pygad.GA(
        num_generations=50,
        sol_per_pop=50,
        num_parents_mating=20,
        fitness_func=fitness_rastrigin_pygad,
        num_genes=12,
        gene_type=float,
        gene_space={'low': -5.12, 'high': 5.12},
        mutation_probability=mut_rate,
        suppress_warnings=True
    )
    ga.run()
    _, fitness, _ = ga.best_solution()
    print(f"Mutation {mut_rate}: {fitness}")

Knapsack Best Fitness: 480.0
Rastrigin Best Fitness: -6.5157585838990855 (Rastrigin value: 6.5157585838990855)

Hyperparameter exploration:
Population 20: 450.0
Population 50: 480.0
Population 100: 480.0
Mutation 0.05: -10.227978936105146
Mutation 0.15: -18.438156570123894
Mutation 0.25: -34.42778412726588
